In [ ]:
import sys
import random

sys.path.append('C:/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner


def path_planner(spawn_points, map):
    """
    Get the longest route waypoints from a starting point to various spawn points.

    Args:
        start_point (carla.Location): The starting location from which to trace routes.
        spawn_points (list[carla.SpawnPoint]): A list of spawn points to which routes will be traced.
        map (carla.Map): The Carla map object, used for route planning.

    Returns:
        list[tuple]: A list of waypoints representing the longest route found,
                     where each waypoint is a tuple containing a Carla waypoint object
                     and its associated location.
    """
    
    def has_lane_change_gap(route, gap_threshold=4):
          
        for i in range(len(route) - 1):
            wp1 = route[i][0]
            wp2 = route[i+1][0]
            
            # 2D distance only (ignoring Z)
            dx = wp2.transform.location.x - wp1.transform.location.x
            dy = wp2.transform.location.y - wp1.transform.location.y
            distance = math.sqrt(dx*dx + dy*dy)
            
            # If distance exceeds threshold, it's likely a lane change
            if distance > gap_threshold:
                
                print(f"Large gap detected: {distance:.2f} meters")
                print(f"  WP1: x={wp1.transform.location.x:.2f}, y={wp1.transform.location.y:.2f}, z={wp1.transform.location.z:.2f}")
                print(f"  WP2: x={wp2.transform.location.x:.2f}, y={wp2.transform.location.y:.2f}, z={wp2.transform.location.z:.2f}")
                print(f"  Lane IDs: {wp1.lane_id} -> {wp2.lane_id}")
                print(f"  Road IDs: {wp1.road_id} -> {wp2.road_id}")
                
                return True
                
        return False
       
    sampling_resolution = 1
    grp = GlobalRoutePlanner(map, sampling_resolution)
    
    # Option 1: Random
    spawn_point = random.choice(spawn_points)
    
    # Option 2: Only Intersections
    # intersections = (0, 99, 21, 59, 79, 51, 52, 102, 11, 12, 25, 24)
    # spawn_point = spawn_points[random.choice(intersections)]
    
    # For Option 1 and 2:
    # Pick the longest possible route
    distance = 0
    for loc in spawn_points:
        
        cur_route = grp.trace_route(spawn_point.location, loc.location)
        
        # Remove first and last points from the route to avoide lane changes in last or first waypoint (common)
        if len(cur_route) > 2:
            cur_route = cur_route[1:-1]
        
        if len(cur_route) > distance:
            distance = len(cur_route)
            route = cur_route
            
    # Option 3: Known start and end point route        
    # spawn_point = spawn_points[2]
    # end_point = spawn_points[51]        
    # route = grp.trace_route(spawn_point.location, end_point.location)
    # route = route[1:-1]
    
    print(has_lane_change_gap(route))
            
    return route, spawn_point


def get_path_waypoints(route):
    """
    Extract the juctions start and end waypoints.

    Args:
        route (list[tuple]): A list of waypoints, where each waypoint is represented 
                             as a tuple containing a Carla waypoint object and its location.

    Returns:
        list[carla.Waypoint]: A list of waypoints
    """
    # List to hold waypoints of different types
    path_waypoints = []
    
    # Flag to track if the last waypoint is a junction
    last_wp_is_junction = route[-1][0].is_junction if route else False

    # Iterate over waypoints in the route
    for i in range(len(route)):
        if route[i][0].is_junction:
            # Check if it's the start of a junction waypoints set
            if (i == 0 or not route[i-1][0].is_junction):  # Start of a junction
                path_waypoints.append(route[i][0])
            # Check if it's the end of a 'j' set
            elif (i == len(route)-1 or not route[i+1][0].is_junction):  # End of a junction
                path_waypoints.append(route[i][0])
                
    # Add the last waypoint if it is not a junction
    if not last_wp_is_junction:
        path_waypoints.append(route[-1][0])

    
    return path_waypoints  # Return the list of waypoints


def update_waypoint(path_waypoints, current_wp_indx, current_loc):
    
    """
    Update to the next waypoint if the vehicle is close enough to the current one.
    
    returns,
        current_wp: next waypoint to reach. (not update if its the destination)
        wp_type: way point type (normal, destination, intersection, other)
        reached: boolean value to know that waypoint has reached
        dist = distance to new waypoint
    
    """
    threshold = 3 # 3m
    current_wp = path_waypoints[current_wp_indx]
    wp_type = 0 # normal by default
    reached = False # current waypoint reached
    destination = False # end of the path
            
    # Calculate distance to the current waypoint
    dist = current_loc.distance(current_wp.transform.location)
    
    # Check if the vehicle is within the threshold distance to update to the next waypoint
    if dist < threshold:
        reached = True
        if current_wp_indx < len(path_waypoints) - 1:
            current_wp_indx += 1  # Move to the next waypoint
            current_wp = path_waypoints[current_wp_indx]  # Update current waypoint
            dist = current_loc.distance(current_wp.transform.location)  # Calculate new distance
            if current_wp_indx == len(path_waypoints) - 1:
                wp_type = 1 # next waypoint is destination
        else:
            destination = True  # Current waypoint is the destination
    return current_wp, current_wp_indx, wp_type, reached, dist, destination


def get_dis_ang_from_nearest_wp(current_loc, vehicle_yaw, route):
    """
    Calculate the distance and angle difference between the vehicle and the nearest waypoint in the route.

    Args:
        vehicle (carla.Vehicle): The vehicle object.
        route (list[tuple]): A list of waypoints from the route, where each entry is a tuple
                             containing a CARLA Waypoint object and a RoadOption.

    Returns:
        tuple: A tuple containing the distance to the nearest waypoint and the absolute angular difference in yaw.
    """
    # Get the vehicle's current location
    selected_wp, dist_to_wp = get_closest_wp(current_loc, route)

    # Check if we found a valid waypoint
    if selected_wp:        
        dir_differ = get_angle(vehicle_yaw, selected_wp.transform.rotation.yaw)

        return dist_to_wp, dir_differ
    
    else:
        print("No waypoint found.")
        return 0.0, 0.0
    

def get_angle(vehicle_yaw, wp_yaw):    
    # Calculate the yaw difference
    dir_differ = vehicle_yaw - wp_yaw

    # Normalize to -180 to 180 degrees
    dir_differ = (dir_differ + 180) % 360 - 180
    return dir_differ

    
def get_closest_wp(current_loc, route):
    min_dist = float('inf')  # Initialize to a very large distance
    closest_wp = None  # To store the closest waypoint
    # Find the closest waypoint in the route
    for wp, _ in route:
        wp_loc = wp.transform.location
        dist = current_loc.distance(wp_loc)
        
        # Update if this waypoint is closer
        if dist < min_dist:
            min_dist = dist
            closest_wp = wp  # Store the closest waypoint object
            
    return closest_wp, min_dist 

In [ ]:
import random
import carla
import math

def draw_route(route, world, lane_change_points=None, seconds=20.0):
    for i, (waypoint, _) in enumerate(route):
        # Draw only every 10th waypoint
        if i % 20 == 0:
            world.debug.draw_box(
                carla.BoundingBox(waypoint.transform.location, carla.Vector3D(0.15, 0.15, 0.15)),
                carla.Rotation(),
                thickness=0.5,
                color=carla.Color(r=0, g=0, b=255),
                life_time=seconds
            )

    print(f"Displayed {len(route)//20} waypoints on the map (every 10th waypoint)!")
    
    # Draw lane change points if provided
    if lane_change_points:
        for idx, (wp1_loc, wp2_loc) in enumerate(lane_change_points):
            # Draw a line between them
            world.debug.draw_line(
                wp1_loc,
                wp2_loc,
                thickness=1,
                color=carla.Color(r=255, g=0, b=0),  # Yellow line
                life_time=seconds
            )
        
        print(f"Displayed {len(lane_change_points)} lane change locations!")


def main():
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
    
    map = world.get_map()
    spawn_points = map.get_spawn_points()
    
    route, spawn_point = path_planner(spawn_points, map)
    
    if route:
        # Find lane change points in THIS route
        lane_change_points = find_lane_change_points(route, gap_threshold=4.0)
        
        # Draw the route with lane change markers
        draw_route(route, world, lane_change_points, seconds=30.0)
    else:
        print("No valid route found.")


def find_lane_change_points(route, gap_threshold=4.0):
    """Find all lane change points in the route and return their locations."""
    lane_change_points = []
    
    for i in range(len(route) - 1):
        wp1 = route[i][0]
        wp2 = route[i+1][0]
        
        dx = wp2.transform.location.x - wp1.transform.location.x
        dy = wp2.transform.location.y - wp1.transform.location.y
        distance = (dx*dx + dy*dy) ** 0.5
        
        if distance > gap_threshold:
            print(f"Lane change found at index {i}: {distance:.2f}m gap")
            print(f"  WP1: x={wp1.transform.location.x:.2f}, y={wp1.transform.location.y:.2f}")
            print(f"  WP2: x={wp2.transform.location.x:.2f}, y={wp2.transform.location.y:.2f}")
            print(f"  Lane IDs: {wp1.lane_id} -> {wp2.lane_id}")
            
            lane_change_points.append((
                wp1.transform.location,
                wp2.transform.location
            ))
    
    return lane_change_points


if __name__ == "__main__":
    main()